In [1]:
import pandas as pd
import numpy as np
#df = pd.read_csv(r'/home/refu0917/lungcancer/seerdb/csv_folder/seer_reg13.csv')
#df = pd.read_csv(r'/home/refu0917/lungcancer/seerdb/csv_folder/seer_reg17.csv',index_col=[0])
df = pd.read_csv(r'/home/refu0917/lungcancer/seerdb/csv_folder/seer_reg17_ver1.csv')
rename_dict = {'Patient ID':'id',
            'Site recode ICD-O-3/WHO 2008':'site',
            'Sequence number':'seq_number',
            'Sex' : 'Gender',
            'Year of diagnosis' : 'diag_year',
            'Age recode with <1 year olds':'Age',
            'CS tumor size (2004-2015)':'TMRSZ',
            'Regional nodes positive (1988+)':'LYMND',
            'Derived AJCC Stage Group, 7th ed (2010-2015)':'AJCCstage',
            'Visceral and Parietal Pleural Invasion Recode (2010+)':'SSF2',
            'Separate Tumor Nodules Ipsilateral Lung Recode (2010+)':'SSF1',
            'Grade (thru 2017)':'DIFF',
            'Survival months':'survival_month',
            'Vital status recode (study cutoff used)': 'vital_sts',
            'ICD-O-3 Hist/behav, malignant': 'histologic',
            'Behavior code ICD-O-3' : 'behavior',
            'Regional nodes examined (1988+)' : 'LYMND_exm',
            'Primary Site - labeled':'primary_site',
            'Survival months flag': 'survival_month_flag',
            'Mets at DX-Other (2016+)': 'SSF4',
            'RX Summ--Surg Prim Site (1998+)':'OP'}
df = df.rename(columns=rename_dict)

df['histologic'] = df['histologic'].apply(lambda x:x[:6])
df['primary_site'] = df['primary_site'].apply(lambda x:x[:5])
df = df[df['survival_month'] !='Unknown']
df['survival_month'] = df['survival_month'].astype('int')
df['survival_month_flag'] = df['survival_month_flag'].apply(lambda x:1 if x == 'Complete dates are available and there are more than 0 days of survival' else 0)
print(len(df))

7697408


In [2]:
df.head()

,id,seq_number,Gender,diag_year,Age,TMRSZ,LYMND,AJCCstage,SSF2,SSF1,DIFF,SSF4,OP,survival_month,histologic,primary_site,survival_month_flag
0,21,2nd of 2 or more primaries,Male,2002,40-44 years,Blank(s),98,Blank(s),Blank(s),Blank(s),Unknown,Blank(s),40,208,9061/3,C62.9,1
1,24,2nd of 2 or more primaries,Male,2000,65-69 years,Blank(s),0,Blank(s),Blank(s),Blank(s),Well differentiated; Grade I,Blank(s),30,49,8140/3,C20.9,1
2,30,2nd of 2 or more primaries,Female,2008,35-39 years,035,1,Blank(s),Blank(s),Blank(s),Moderately differentiated; Grade II,Blank(s),30,143,8140/3,C19.9,1
3,43,2nd of 2 or more primaries,Female,2008,60-64 years,090,98,Blank(s),Blank(s),Blank(s),Well differentiated; Grade I,Blank(s),60,0,8801/3,C38.1,1
4,52,2nd of 2 or more primaries,Male,2008,85+ years,999,98,Blank(s),Blank(s),Blank(s),Unknown,Blank(s),31,26,8742/3,C44.4,1


In [3]:
ipc = df[(df.seq_number == 'One primary only') | (df.seq_number == '1st of 2 or more primaries')]
spm = df[df.seq_number == '2nd of 2 or more primaries']
print(len(ipc), len(spm))

ipc = ipc.loc[ipc['histologic'].isin(['8012/3', '8046/3', '8070/3', '8140/3', '8240/3', '8250/3', '8560/3', '9053/3'])]
print(len(ipc))
ipc = ipc.loc[ipc['primary_site'].isin(['C33.9', 'C34.0', 'C34.1', 'C34.2', 'C34.3', 'C34.8' ,'C34.9'])]
print(len(ipc))
ipc = ipc[ipc['survival_month_flag'] == 1]
print(len(ipc))

6280333 1152346
2339943
467904
448205


In [4]:
'''ipc_flwup = ipc[ipc['survival_month'] !='Unknown']
ipc_flwup['survival_month'] = ipc_flwup['survival_month'].astype('int')
ipc_flwup = ipc_flwup[ipc_flwup['survival_month']>=48]
print(len(ipc_flwup))
'''
#spm = spm[spm['survival_month'] !='Unknown']
#spm['survival_month'] = spm['survival_month'].astype('int')
spm = spm[spm.id.isin(ipc.id.tolist())]
spm = spm[spm.survival_month >=6]
print(len(spm))

22952


In [5]:
rule1 = spm.loc[~spm['primary_site'].isin(['C33.9', 'C34.0', 'C34.1', 'C34.2', 'C34.3', 'C34.8' ,'C34.9'])]
print(len(rule1))
spm = spm.loc[spm['primary_site'].isin(['C33.9', 'C34.0', 'C34.1', 'C34.2', 'C34.3', 'C34.8' ,'C34.9'])]
print(len(spm))

12036
10916


In [6]:
spm['comp_histologic'] = ipc.loc[ipc.id.isin(spm.id.tolist())]['histologic'].tolist()
spm['comp_diagyear'] = ipc.loc[ipc.id.isin(spm.id.tolist())]['diag_year'].tolist()

rule2 = spm.loc[(spm['histologic']!=spm['comp_histologic'])]
spm = spm.loc[(spm['histologic']==spm['comp_histologic'])]
print(len(rule2), len(spm))
rule3 = spm.loc[(spm['diag_year']-spm['comp_diagyear'])>=2]
spm = spm.loc[(spm['diag_year']-spm['comp_diagyear']<2)]
print(len(rule3), len(spm))

6630 4286
2478 1808


In [7]:
rule2 = rule2.drop(columns=['comp_histologic'])
rule3 = rule3.drop(columns=['comp_diagyear'])
spc = pd.concat([rule1, rule2, rule3], axis = 0)
ipc['Class'] = 0
ipc.loc[ipc[ipc.id.isin(spc.id.tolist())].index,"Class"] = 1
ipc.Class.value_counts()

0    427061
1     21144
Name: Class, dtype: int64

In [8]:
ipc['Age'] = ipc['Age'].apply(lambda x:int(x.split('-')[0][0]) if x[:2] != '85' else int(8))
ipc = ipc.loc[~ipc['TMRSZ'].isin([888, 990, 996, 997, 998])]
ipc = ipc.loc[~ipc['LYMND'].isin([95])]
ipc = ipc.loc[~ipc['AJCCstage'].isin(['OCCULT'])]

In [9]:
ipc.SSF4.value_counts()
SSF4_dict = {'generalized metastases such as carinomatosis' : 0,
             'Not applicable' : 0,
             'None; no other metastases' : 1,
             'Yes; distant mets in known site(s) other than bone, brain, liver, lung, dist LN' : 2,
             'Unknown' : 9,
             'Blank(s)' : 999
            }

In [10]:
ipc[ipc.SSF4 == 'Yes; distant mets in known site(s) other than bone, brain, liver, lung, dist LN'].Class.value_counts()

0    15341
1      123
Name: Class, dtype: int64

In [11]:
def op_convert(x):
    if x == 0:
        return 1
    if x == 99:
        return 9
    if x != 0 and x != 99:
        return 2

def regional_node_encode(x):
    if x == 'Blank(s)':
        return 999
    elif x == 0 or x == 98:
        return 1
    elif x == 99 or x == 97:
        return 9
    elif 1<=x<=2:
        return 2
    elif 3<=x<=6:
        return 3
    elif 7<=x<=15:
        return 4
    elif 16<=x:
        return 5

def tumor_encode(size):
    if size == 'Blank(s)':
        return 999
    size = int(size)
    if size == 999:
        return 9
    elif size <= 49:
        return 1
    elif 50 <= size <= 99:
        return 2
    elif 100 <= size <= 149:
        return 3
    else:
        return 4

SSF2_dict = {'PL0; No evidence; Tumor does not completely traverse the elastic layer of pleura':1,
                    'Tumor extends to pleura, NOS; not stated if visceral or parietal':2,
                    'PL1 or PL2; Invasion of visceral pleura present, NOS':2,
                    'PL3; Tumor invades into or through the parietal pleura OR chest wall':2,
                    'Not documented; No resection of primary; Not assessed or unknown if assessed':9,
                    'Blank(s)':999}

SSF1_dict = {'None; No intrapulmonary mets; Foci in situ/minimally invasive adenocarcinoma':1,
                    'Separate nodules of same hist type in ipsilateral lung, same lobe':2,
                    'Separate nodules of same hist type in ipsilateral lung, different lobe':2,
                    'Separate tumor nodules, ipsilateral lung, unknown if same or different lobe':2,
                    'Separate nodules of same hist type in ipsilateral lung, same AND different lobes':2,
                    'Not documented; Primary tumor is in situ; Not assessed or unknown if assessed':9,
                    'Blank(s)':999}

grade_map_dict = {'Moderately differentiated; Grade II':2,
                  'Poorly differentiated; Grade III':3,          
                  'Well differentiated; Grade I':1,             
                  'Undifferentiated; anaplastic; Grade IV':4,
                  'Unknown':9,
                  'Blank(s)':999}

SSF4_dict = {'generalized metastases such as carinomatosis' : 0,
             'Not applicable' : 0,
             'None; no other metastases' : 1,
             'Yes; distant mets in known site(s) other than bone, brain, liver, lung, dist LN' : 2,
             'Unknown' : 9,
             'Blank(s)' : 999
            }

gender_dict = {'Male':1, 'Female':2}

ipc['Gender'] = ipc['Gender'].map(gender_dict)
ipc['LYMND'] = ipc['LYMND'].map(regional_node_encode)
ipc['TMRSZ'] = ipc['TMRSZ'].map(tumor_encode)
ipc['OP'] = ipc['OP'].map(op_convert)
ipc['SSF4'] = ipc['SSF4'].map(SSF4_dict)
ipc['SSF2'] = ipc['SSF2'].map(SSF2_dict)
ipc['SSF1'] = ipc['SSF1'].map(SSF1_dict)
ipc['DIFF'] = ipc['DIFF'].map(grade_map_dict)
ipc['AJCCstage'] = ipc['AJCCstage'].replace('UNK Stage',9)
ipc['AJCCstage'] = ipc['AJCCstage'].replace('Blank(s)',999)

for i,j in zip(['V','III', 'II','I'], [4,3,2,1]):
    index = ipc[ipc['AJCCstage'].str.contains(i,na=False)].index
    ipc.loc[index,'AJCCstage'] = ipc[ipc['AJCCstage'].str.contains(i, na=False)]['AJCCstage'].apply(lambda x:j)

ipc = ipc.replace(999, np.nan)
ipc = ipc.drop(columns=['id','seq_number', 'primary_site', 'histologic', 'survival_month', 'survival_month_flag'])

In [12]:
ipc.head()

,Gender,diag_year,Age,TMRSZ,LYMND,AJCCstage,SSF2,SSF1,DIFF,SSF4,OP,Class
274,2,2000,6,NaN,1,NaN,NaN,NaN,9.0,NaN,2,0
318,1,2000,8,NaN,1,NaN,NaN,NaN,9.0,NaN,1,0
325,2,2001,4,NaN,1,NaN,NaN,NaN,9.0,NaN,1,0
409,1,2000,7,NaN,1,NaN,NaN,NaN,3.0,NaN,1,0
451,1,2000,5,NaN,1,NaN,NaN,NaN,3.0,NaN,1,0


In [12]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge, Ridge
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

imputer = IterativeImputer(random_state=0, estimator=RandomForestClassifier(),initial_strategy = 'most_frequent')
temp_imp = imputer.fit_transform(temp)

/home/refu0917/.local/lib/python3.8/site-packages/sklearn/impute/_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)
/home/refu0917/.local/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [39]:
temp.DIFF.value_counts()

9.0    76226
3.0    46993
2.0    32462
1.0    10212
4.0     1573
Name: DIFF, dtype: int64

In [14]:
temp.isnull().sum()

Gender            0
diag_year         0
Age               0
TMRSZ         78163
LYMND             0
AJCCstage     78163
SSF2              0
SSF1              0
DIFF          37688
SSF4         126991
OP                0
Class             0
dtype: int64

In [15]:
temp.TMRSZ.value_counts()

1.0    68810
2.0    31789
9.0    21530
3.0     3636
4.0     1226
Name: TMRSZ, dtype: int64

In [16]:
a = pd.DataFrame(temp_imp,columns=temp.columns)
a.TMRSZ.value_counts()

1.0    118623
2.0     50083
9.0     30192
3.0      4650
4.0      1606
Name: TMRSZ, dtype: int64

In [58]:
temp.loc[temp[temp == 999].count(axis=1) != 1]['diag_year'].value_counts()

2016    20405
2017    20088
2018    18907
2019    18795
2011       14
2010       11
2013       10
2015        9
2012        8
2014        6
Name: diag_year, dtype: int64

In [127]:
ipc.loc[ipc[ipc == 999].count(axis=1) == 2]['diag_year'].value_counts().sort_index()

2016    20395
2017    20080
Name: diag_year, dtype: int64

In [152]:
df4 = pd.read_csv(r'/home/refu0917/lungcancer/server/AllCaseCtrl_final.csv')
df4['Class'] = df4['Class'].apply(lambda x:1 if x != 0 else 0)
columns = ["Class","LOC", "Gender", "Age", "AJCCstage", "DIFF", "LYMND", "TMRSZ", "SSF1", "SSF2", "SSF4", "FullDate"] #
df4 = df4[columns]
df4['year'] = [int(x[:4]) for x in list(df4['FullDate'])]

#df4_2010 = df4[columns]


In [94]:
df4.isna().sum(axis = 1).value_counts().sort_index()

0    6977
1    1559
2      27
3    5322
4      27
dtype: int64

In [154]:
df4 = df4[df4.year >= 2010]

In [168]:
df4.isna().sum(axis=1).value_counts()

0    6977
1    1332
3     753
2      11
4       1
dtype: int64

In [167]:
df4[df4.isna().sum(axis=1) == 3]['Class'].value_counts()

0    732
1     21
Name: Class, dtype: int64

In [76]:
ipc_no_null = ipc[ipc!=999]
ipc_no_null = ipc_no_null.dropna()
ipc_no_null.Class.value_counts()
ipc_no_null['LOC'] = 9

In [87]:
ipc_no_null.to_csv(r'/home/refu0917/lungcancer/data/seerdb.csv')

In [68]:
from tableone import TableOne, load_dataset
#drawe table one

columns = ["Class","Gender", "Age",
        "AJCCstage", "DIFF", "LYMND", "TMRSZ",
        "SSF1", "SSF2"]
groupby = ["Class"]
categorical = ["Class","Gender", "Age",
        "AJCCstage", "DIFF", "LYMND", "TMRSZ",
        "SSF1", "SSF2"]
seer_tableone = TableOne(ipc_no_null, columns=columns,categorical=categorical)
seer_tableone.to_csv('seer_tableone.csv')